# Segmenting and Clustering Neighborhoods in Toronto Assignment

## First Importing Librarys

In [1]:
import requests        #For HTTP Requests
import lxml.html as lh #For Html to ? Convertion
import pandas as pd    #For Creating Dataframs
from lxml import etree #For Converting HTML Elements to Raw HTML
import numpy as np     #For Vectorized Computation

## Second Preparing Data

Getting a data from a table located somewhere in a web page isn't that difficult, In order to get data then there is couple of things you need to know about HTML.
First HTML is a markup language which means it's written as tags like: 

and those tags always have a start and end so they look like something like this:  

The other thing is these tags sometimes contains attributes that defines some property's for the tag, A tag with an attribute will look like something like this:

There is two attributes that we need to focus on:
<ul>
    <li><strong>Class</strong> : Which defines the style class of the tag (Same Class Can Be Used For Multiple tags)</li>
    <li><strong>Id</strong>    : Which defines a unique name for the tag (Ids are Unique and Can't Be Used For Multiple tags)</li>
</ul>
So in order to get a tag then the first thing that you need to consider is to find the id of the tag (<font style='color:#f90909 !important;'>IF EXISTED</font>), and in order to get that tag is by opening your browser and 'Right Click' on that tag and click on 'Inspect' as follow:

<img src="Ex1.png" />

After that a small screen will appear and it will contain the HTML of the page and focus on the tag that you have 'Right-Clicked' on it, you will need to look for a 'Table' tag and try to find if there an id attribute with it.

<img src="Ex2.png" />

Based on the above image you can see that there is't an id with the table 'BUT' there is a class attribute so we will simply copy the value of that class and  assign it to a new variable in our notebook

In [17]:
Table_Class = "wikitable sortable" #Just Copy the First Two Words

### Data Enquiring
Now we will get the contents of the page and dig for that Table and for that we will gonna use LXML library

In [19]:
wikipedia_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  #Page Url
page = requests.get(wikipedia_url)                                               #Get the Page
doc = lh.fromstring(page.content)                                                #getting the contents of the website

#Now we will get the wikipedia table that has a class attribute thet contints "wikitable sortable"
results  = doc.xpath("//table[@class='%s']" % Table_Class )

table = results[0]          #Getting The Table Which Should be the First in List (as HTML Element)
raw = etree.tostring(table) #Getting Raw HTML From HTML Element
df = pd.read_html(raw)[0]   #Use Pandas 'read_html' Function to Convert From HTML to Dataframe

df.head()                   #Printing Result Dataframe

<Response [200]>


### Cleaning Data
First we will delete every row that has 'Borough' value is not assign, then we will copy the value of 'Borough' column to 'Neighbourhood' that has no value and after that we will reset the dataframe index.
Next is kind of tricky; we will group data based on 'Postcode' and aggregate the child value into one cell but separate them by a comma  

In [14]:
df.drop(df.loc[df['Borough'] == "Not assigned"].index, inplace=True)         #Removing Rows That Has "Not assigned" Value
df.loc[df['Neighbourhood'] == "Not assigned" , 'Neighbourhood'] = df['Borough'];
df = df.reset_index(drop=True)                                               #Reseting Index
tmp = df.groupby(['Postcode'])['Neighbourhood'].agg(', '.join).reset_index() #Grouping Data
tmp['Borough'] = df.groupby(['Postcode'])['Borough'].first().reset_index()['Borough']
df = tmp                                 
df.shape

(103, 3)